## Cost of Capital
##### A notebook by __Matt Harrington (@conda_env)__ for identifying optimal yield farm allocations.

##### Fetch the data from the API and prints the last vault data

In [3]:
import requests
import json
from typing import Union, List

LIST_VAULTS_URL = 'https://api.vaults.fyi/v1/vaults'
API_KEY_HEADERS = {'X-API-Key': 'a9cmW7qWgb8fIg4pMyvRaMgDIsFWWeSpOh6ljM7uuuI'}

def get_vault_data(
    tokens: Union[str, List[str]] = 'USDC',
    network: Union[str, List[str]] = 'mainnet',
    tvl_min: int = 1_000_000
) -> List[dict]:
    ret = []
    tokens_to_pull = [tokens] if isinstance(tokens, str) else tokens
    networks_to_pull = [network] if isinstance(network, str) else network
    for t in tokens_to_pull:
        for n in networks_to_pull:
            params = { 
                'token' : t,
                'network' : n,
                'tvl_min' : tvl_min
            }
            response = requests.get(
                LIST_VAULTS_URL,
                headers=API_KEY_HEADERS,
                params=params
            )
            if response.status_code == 200:
                ret.append(response.json())
            else:
                raise Exception(f"Failed to fetch {t}@{n} data: {response.status_code}")
    return ret

# Fetch data from the API
vault_data = get_vault_data()

# We can print out the vault_data or inspect it to understand the structure and proceed
print(json.dumps(vault_data[-1], indent=2))

[
  {
    "address": "0x1B19C19393e2d034D8Ff31ff34c81252FcBbee92",
    "network": "mainnet"
  },
  {
    "address": "0x39AA39c021dfbaE8faC545936693aC917d5E7563",
    "network": "mainnet"
  },
  {
    "address": "0x530824DA86689C9C17CdC2871Ff29B058345b44a",
    "network": "mainnet"
  },
  {
    "address": "0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c",
    "network": "mainnet"
  },
  {
    "address": "0xA1F3aca66403D29b909605040C30ae1F1245d14c",
    "network": "mainnet"
  },
  {
    "address": "0xBcca60bB61934080951369a648Fb03DF4F96263C",
    "network": "mainnet"
  },
  {
    "address": "0xCA30c93B02514f86d5C86a6e375E3A330B435Fb5",
    "network": "mainnet"
  },
  {
    "address": "0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE",
    "network": "mainnet"
  },
  {
    "address": "0xc3d688B66703497DAA19211EEdff47f25384cdc3",
    "network": "mainnet"
  },
  {
    "address": "0xdd50C053C096CB04A3e3362E2b622529EC5f2e8a",
    "network": "mainnet"
  },
  {
    "address": "0xfe119e9C24ab79F1bDd5dd884B

In [ ]:
## Confirm I'm grabbing what I want to
from pprint import pprint
print(len(vault_data))
counter = 0
for v in vault_data:
    if v['network'] == 'mainnet':
        counter += 1
print(counter)

#### Define portfolio constraints

In [5]:
# Constants
TAO = 1000000  # Total capital to be allocated
NETWORKS = ['mainnet']  # Networks to consider
TOKENS = ['USDC', 'USDT']  # Tokens to consider
SLIPPAGE = 0.0001  # Slippage rate
MAX_VAULTS = 5  # Maximum number of vaults to allocate to
MAX_TVL_SHARE = 0.1  # Maximum share of TVL for allocation
HOLDING_DAYS = 30 # Number of days to hold the capital

# # Static API endpoint
# API_ENDPOINT = "https://api.vaults.fyi/v1/vaults?token=USDC&network=mainnet&tvl_min=1000000"

### Optimize the allocation of capital across the vaults

Use cvxpy.py to perform the convex optimization along the yield curves.

In [ ]:
import cvxpy as cp
import numpy as np
from pprint import pprint

# Fetch data from the API
vault_data = get_vault_data()

# Number of vaults
num_vaults = len(vault_data)

# Total Value Locked for each vault
tvls = np.array([vault['tvlInUsd'] for vault in vault_data])

# Optimization variables
allocations = cp.Variable(num_vaults)
selection = cp.Variable(num_vaults, boolean=True)

# Yield function for each vault (placeholder)
yields = cp.multiply(allocations, np.array([vault['apy'] for vault in vault_data]))

# Slippage adjusted yields
adjusted_yields = yields * (1 - SLIPPAGE)

# Objective: Maximize the total adjusted yield
objective = cp.Maximize(cp.sum(adjusted_yields))

# Constraints
constraints = [
    cp.sum(allocations) <= TAO,  # Total allocation must be TAO
    allocations <= selection * (0.5 * TAO),  # No more than half of TAO to any individual pool
    allocations >= 0,  # Allocations must be non-negative
    cp.sum(selection) <= MAX_VAULTS,  # No more than MAX_VAULTS vaults
    allocations <= tvls * MAX_TVL_SHARE,  # Allocation must be less than 10% of the TVL
]

# Problem
problem = cp.Problem(objective, constraints)

# Solve the problem using a solver that supports integer/binary programming
problem.solve(solver=cp.CBC)

# Check the status of the solution
if problem.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("Problem status:", problem.status)
else:
    # Optimal allocations
    optimal_allocations = allocations.value

allocated_vaults = np.where(optimal_allocations > 0)[0]

# Display the results
print("Optimal allocations:")
pprint({f"{vault_data[i]['name']}": 
    f"${optimal_allocations[i]/1_000:.2f}k" for i in allocated_vaults})

### Compose data & metadata into a single data table

In [ ]:
import pandas as pd

n_allocations = len(allocated_vaults)

# Convert allocations to a numpy array for easier handling
optimal_allocations_array = np.array(optimal_allocations)

# Calculate the anticipated APY for each allocated vault
post_deposit_apys = []
for i in allocated_vaults:
    tvl_v = vault_data[i]["tvlInUsd"]
    apy_v = vault_data[i]["apy"] / 100
    allocation_v = optimal_allocations_array[i]
    adjusted_apy_v = apy_v * (tvl_v / (tvl_v + allocation_v))
    post_deposit_apys.append(adjusted_apy_v)

# Calculate the net APY for the portfolio
net_apy = np.average([
        post_deposit_apys[i] / 100 for i in range(n_allocations)
    ], weights=optimal_allocations_array[allocated_vaults])

# Calculate the expected USD return for each vault allocation
expected_yearly_usd_return = net_apy * sum(optimal_allocations_array[allocated_vaults])

# Prepare the data for the table
table_data = {
    "Protocol Name": [vault_data[i]["protocolName"] for i in allocated_vaults],
    "Network": [vault_data[i]["network"] for i in allocated_vaults],
    "Current APY": [f'{vault_data[i]["apy"] / 100}%' for i in allocated_vaults],
    "Post-Entry APY*": [f'{round(post_deposit_apys[i], 1)}%' for i in range(n_allocations)],
    "Asset Symbol": [vault_data[i]["asset"]["symbol"] for i in allocated_vaults],
    "Vault Allocation": [f'${round(optimal_allocations_array[i] / 1_000, 2)}k' for i in allocated_vaults],
    "Current TVL (USD)": [f'${round(vault_data[i]["tvlInUsd"] / 1_000_000, 2)}mm' for i in allocated_vaults],
    "URL Link": [vault_data[i]["lendLink"] for i in allocated_vaults],
    "Vault Address": [vault_data[i]["address"] for i in allocated_vaults],
}

# Create a DataFrame to display the table
vaults_table = pd.DataFrame(table_data)

# Print the expected USD return and net APY for the portfolio
print(f"Expected {HOLDING_DAYS}d USD return on the ${TAO / 1_000_000}mm " + 
    f"portfolio: ${round(expected_yearly_usd_return * (HOLDING_DAYS / 365) / 1_000, 2)}k")
print(f"Net APY for the portfolio: {round(net_apy * 100, 2)}%")

# Display the table
vaults_table